# Datos climáticos. Tratamiento e incorporación.

En este notebook se limpian los datos climáticos de outliers y, posteriormente, se unen a los datos de importación.

In [1]:
import pandas as pd
import glob
from pathlib import Path

In [2]:
countries = ['austria','belgium','bulgaria','croatia','cyprus','czechia','denmark','estonia','finland','france','germany','greece','hungary',
             'ireland','italy','latvia','lithuania','luxembourg','malta','netherlands','poland','portugal','romania','slovenia','spain','sweden']

In [3]:
dataframes_pre = [pd.read_csv(f'./Climate_data/pre_{country}.csv') for country in countries]
dataframes_temp = [pd.read_csv(f'./Climate_data/temp_{country}.csv') for country in countries]

In [4]:
# Detectamos los outliers de precipitación por encima de valores razonables
for i in range(len(dataframes_pre)): 
    if dataframes_pre[i]['p_mes'].max() > 400:
        print(dataframes_pre[i]['p_mes'].max(), i)

for i in range(len(dataframes_pre)): 
    if dataframes_pre[i]['p_mes'].min() < 0:
        print(dataframes_pre[i]['p_mes'].min(), i)

438.0 0
439.5 3
524.0 9
403.4 11
414.5 13
1338.5 14
421.9 21
438.0 23
440.0 24


In [5]:
# Dado que se incluyen regiones montañosas muy lluviosas damos por buenos todos los outliers salvo el de 1338.5 que lo eliminamos.
for i in range(len(dataframes_pre)):
   dataframes_pre[i] = dataframes_pre[i][dataframes_pre[i]['p_mes'] <= 600]

In [6]:
# Detectamos outliers de temperatura por debajo o por encima de valores razonables (-50, 50)
for i in range(len(dataframes_temp)):
    if dataframes_temp[i]['temp_mes'].min() < -50:
        print(dataframes_temp[i]['temp_mes'].min(), i)
    
for i in range(len(dataframes_temp)): 
    if dataframes_temp[i]['temp_mes'].max() > 50:
        print(dataframes_temp[i]['temp_mes'].max(), i)


-9.96920996838687e+36 9
-9.96920996838687e+36 14
-9.96920996838687e+36 20
-9.96920996838687e+36 21
-9.96920996838687e+36 24
-9.96920996838687e+36 25
52.4 24


In [7]:
# El valor máximo es razonable así que se deja. Los valores claramente anómalos los eliminamos.
for i in range(len(dataframes_temp)):
   dataframes_temp[i] = dataframes_temp[i][dataframes_temp[i]['temp_mes'] >= -50]

In [8]:
# Guardamos los dataframes limpios
for i, df in enumerate(dataframes_pre):
    country = countries[i]
    df.to_csv(f'./Climate_data_limpios/pre_{country}.csv', index=False)

for i, df in enumerate(dataframes_temp):    
    country = countries[i]
    df.to_csv(f'./Climate_data_limpios/temp_{country}.csv', index=False)

Ahora incorporamos los datos climáticos al dataset con las importaciones de cítricos de cada país de la UE.

In [9]:
dataframes_pre = [pd.read_csv(f'./Climate_data_limpios/pre_{country}.csv') for country in countries]
dataframes_temp = [pd.read_csv(f'./Climate_data_limpios/temp_{country}.csv') for country in countries]
dataframes_country = [pd.read_csv(f'./Countries_citricos/{country.capitalize()}.csv') for country in countries]

results =[]

for i in range(len(dataframes_pre)):
    df_pre = dataframes_pre[i]
    df_temp = dataframes_temp[i]
    df_country = dataframes_country[i]

    df_pre = df_pre.groupby(['agno','mes'])['p_mes'].mean().round(3).reset_index()
    df_temp = df_temp.groupby(['agno','mes'])['temp_mes'].mean().round(3).reset_index()

    df_country = df_country.drop(columns=['End date','Marketing Year','Procedure','Preference'])
    df_country['Start date'] = pd.to_datetime(df_country['Start date'])
    df_country['mes'] = df_country['Start date'].dt.month
    df_country['agno'] = df_country['Start date'].dt.year

    df = df_country.merge(df_pre, on=['agno', 'mes'], how='left')
    df = df.merge(df_temp, on=['agno', 'mes'], how='left')


    results.append(df)

El resultado final lo guardamos en un .csv por cada país. 

In [10]:
for i in range(len(results)):
    country = results[i]['Member State'].unique()[0]
    results[i].to_csv(f'./Countries_citricos_clima/{country}.csv', index= False)
    